In [1]:
def readFile(url):
    f = open(url, 'r', encoding='utf-8')
    content = f.read()
    # print(content)
    f.close()
    return content

In [2]:
def correctLine(line, count):
    correctLine = line
    # 修正错误的8
    count8Index = str(count).find('8')
    lineError8Index = line.find('&')
    if (count8Index != -1 and lineError8Index != -1):
        correctLine = line.replace('&', '8')
        print('修正错误的8:', line, '=>', correctLine)
        
    condition1 = bool(re.match('\d{1,3}\.', line))
    condition2 = bool(re.match('\d{1,3}\•', line))
    
    if (condition1 == False and condition2 == False):
        countLen = len(str(count))
        if (line[0: countLen] == str(count)): # line以count开头
            print('没找到.或• 但匹配到题号', count)
            correctLine = str(count) + '.' + line[countLen:]
        elif (line[1: countLen+1] == str(count)):
            print('没找到.或• 但匹配到题号，且题号前面有个空格', count)
            correctLine = str(count) + '.' + line[countLen+1:]
    return correctLine

In [3]:
def isQuestionStartLine(line, curCount):
    condition1 = bool(re.match('\d{1,3}\.', line))
    condition2 = bool(re.match('\d{1,3}\•', line))
    condition3 = False

#     line = correctLine(line, curCount)
    
    if (curCount and condition1 == False and condition2 == False):
        countLen = len(str(curCount))
        if (line[0: countLen] == str(curCount)): # line以curCount开头
            print('没找到.或• 但匹配到题号', curCount)
            condition3 = True
#         elif (has8 and lineHasError8 and str(curCount).replace('8',))

    if (condition1 or condition2 or condition3):
        return True
    else:
        return False

In [4]:
import re
# 通过判断newFileUrl文件中的最后一行问题是否在文件中来判断这个newFileUrl是否已经输出过了
def isExistInFile(newFileUrl, outputUrl):
    fdest = open(newFileUrl, 'r', encoding='utf-8')
    newContent = 'unknow'
    for content in fdest.readlines():
#         pattern = re.compile(r'\d{1,3}\.')
#         res = pattern.match(content.strip()) # 问题在第一行不能匹配？？？ 是因为有些第一行有空字符串
#         if (res != None or res1 != None):
#             newContent = content # 获取问题行
#             # break # 注释掉break获取最后一行问题
        if (isQuestionStartLine(content, 0)):
            newContent = content # 获取问题行
            # break # 注释掉break获取最后一行问题
    fdest.close()
    
    if (newContent == 'unknow'):
        print('文件内容出错')
        return 'error'
    
    # print('newContent', newContent)
    f = open(outputUrl, 'r', encoding='utf-8')
    isExist = False
    # print(newContent, f.readlines())
    for line in f.readlines():                          #依次读取每行
        if newContent in line:
            print('已存在', newContent)
            isExist = True
            break
    f.close()
    if isExist:
        return 'exist'
    else:
        return 'not exist'

In [5]:
def isQuestionEndLine(line):
    return bool(re.match('正确答案', line))

In [6]:
def isEmptyLine(line):
    return bool(re.match('\n', line))

In [7]:
def getQuestion(url):
    f = open(url, 'r+', encoding='utf-8')
    retrieving = False
    result = ''
    firstQ = ''
    lastQ = ''
    
#     line = f.readline()
#     print ("读取的数据为: %s" % (line))
#     f.seek(0, 0)
#     line = f.readline()
#     print ("读取的数据为: %s" % (line))
    count = 0
    curCount = 0

    for line in f.readlines():                          #依次读取每行
        line = correctLine(line, count)
        if isQuestionStartLine(line, count):
            print('question:', line[0:7])

            curIndex = line.find('.')
            if (curIndex == -1):
                curIndex = line.find('•')
            
            if (curIndex != -1 and int(line[0:curIndex]) == 1):
                count = 1
                curCount = 1
            elif (count == 0):
                index = line.find('.')
                if (index == -1):
                    index = line.find('•')
                count = int(line[0:index])
                curCount = int(line[0:index])
            elif (count < 10):
                curCount = int(line[0:1])
            elif (count < 100):
                curCount = int(line[0:2])
            else:
                curCount = int(line[0:3])
            
            if (count != 0 and curCount != count):
                print('异常行==============', curCount, count)
            
            count = count + 1
            
            if firstQ == '':
                firstQ = line
            lastQ = line
            retrieving = True
        if isQuestionEndLine(line):
            retrieving = False

        if retrieving == True:
            result = result + line
        
        if retrieving == False:
            result = result + '\n'
        
    print('文件名', url[-9: ])
    print('第一行问题：', firstQ[0:6])
    print('最后一行问题：', lastQ[0:6])

    f.close()

    return result

In [8]:
def addQuestionToFile(content, destUrl):
    fo = open(destUrl, "a+") # 打开一个文件用于追加读写
    str = content
    # 在文件末尾写入一行
    fo.seek(0, 2)
    line = fo.write( str )
    # 关闭文件
    fo.close()

In [9]:
def writeFile(content, destUrl):
    fo = open(destUrl, "w+") # 打开一个文件用于读写
    # 在文件末尾写入一行
    fo.seek(0, 0)
    line = fo.write( content )
    # 关闭文件
    fo.close()

In [10]:
def addEmptyLine(destUrl):
    # 打开文件
    fo = open(destUrl, "r+")
    print ("文件名为: ", fo.name)

    line = fo.readline()
    print ("读取的数据为: %s" % (line))

    # 重新设置文件读取指针到开头
    fo.seek(0, 0)
    line = fo.readline()
    fo.seek(0, 0)
    # fo.writelines(['\n'])
    # line = fo.write('555\n')
    print ("读取的数据为: %s" % (line))


    # 关闭文件
    fo.close()

In [11]:
def removeBlankLines(url, maxBlankLines): # 最多保留多少空行
    f = open(url, 'r+', encoding='utf-8')
    count = 0
    prevLineIsEmpty = False
    result = ''
    for line in f.readlines():                          #依次读取每行
        if isEmptyLine(line):
            if prevLineIsEmpty == False:
                result = result + '\n'
            if prevLineIsEmpty == True:
                count = count + 1
            prevLineIsEmpty = True
        else:
            count = 0
            prevLineIsEmpty = False
            result = result + line

    f.close()
    return result

In [12]:
def isAnswerLine(line):
    return bool('正确答案' in line)

In [13]:
# 有些正确答案跟选项在一起，将正确答案单独一行
def newLineForAnswer(url):
    f = open(url, 'r+', encoding='utf-8')
    count = 0
    prevLineIsEmpty = False
    result = ''
    count = 0
    for line in f.readlines():                          #依次读取每行
#         if (count == 0 and line != '\n'):
#             count = count + 1
#             result = '\n'
        
        if isAnswerLine(line):
            if (line[0:4] != '正确答案'):
                print('正确答案单独一行:', line[0:4], line)
                index = line.find('正确答案')
                result = result + line[0:index] + '\n' + line[index:]
            else:
                result = result + line
        else:
            result = result + line

    f.close()
    return result

In [14]:
def convertFile(originalUrl, outputUrl):
    # addEmptyLine(originalUrl)
    isExist = isExistInFile(originalUrl, outputUrl)
    # questions1 = getQuestion(originalUrl)
    # print('questions1', questions1)
    # addQuestionToFile(questions1, outputUrl)
    # reault = readFile(outputUrl)
    # print(reault)
    # result1 = removeBlankLines(outputUrl, 1)
    # print('result', result1)

    # 没有写入过，将该文件的内容写入outputUrl中
    if (isExist == 'not exist'):
        questions = getQuestion(originalUrl)
        addQuestionToFile(questions, outputUrl)
        result = readFile(outputUrl)
        return result
    else:
        print('isExist：', originalUrl)
        return ''

In [15]:
# 获取所有需要文字识别的文件路径，文件路径为 imgaeBaseUrl + namePrefix + [startIndex, endIndex] + '.txt'
# eg: /Users/apple/zyc/zhl/旅游管理题目/5/5-txt/0_01.txt
def identifyAllFile(baseUrl, namePrefix, startIndex, endIndex):
    urls = []
    for index in range(startIndex, endIndex+1):
        url = baseUrl + namePrefix
        if index < 10:
            index = '0' + str(index)
        url = url + str(index) + '.txt'
        urls.append(url)
    return urls

In [16]:
# urls = identifyAllFile('/Users/apple/zyc/zhl/旅游管理题目/5/5-txt/', '0_', 1, 81)
urls = ['/Users/apple/zyc/zhl/旅游管理题目/6/6-txt/all.txt']
tmpUrl = '/Users/apple/zyc/zhl/旅游管理题目/6/output/6-tmp.txt'
outputUrl = '/Users/apple/zyc/zhl/旅游管理题目/6/output/6-result.txt'
# 前置处理
# 将正确答案单独一行 并且 检查每个文件第一个字符串是否是空字符

tmpData = ''
for url in urls:
    nd = newLineForAnswer(url) # 将正确答案单独一行 并且 检查每个文件第一个字符串是否是空字符
    tmpData = tmpData + nd + '\n'
writeFile(tmpData, tmpUrl)        

writeFile('', outputUrl) # 清空输出文件
convertFile(tmpUrl, outputUrl)     
    
# 去掉空白行
data = removeBlankLines(outputUrl, 1)
writeFile(data, outputUrl)

question: 1.《旅游法》
没找到.或• 但匹配到题号 2
question: 2.根据《民法
question: 3.依据《民法
question: 4.《民法典》
question: 5.《旅行社条
question: 6.《旅行社条
question: 7.《导游管理
question: 8.《导游管理
question: 9.根据《消费
question: 10.旅行社准
question: 11.根据《旅
question: 12.根据《自
question: 13.人民民主
question: 14.升挂国旗
question: 15.人民英雄
question: 16.《关于促
question: 17.2021
question: 18.2021
question: 19.2021
question: 20.2021
question: 21.2016
question: 22.在处理突
question: 23.衬衫的袖
question: 24.取得高级
question: 25.地陪应在
question: 26.如果陪同
question: 27.在第一站
question: 28.领队与旅
question: 29.若导游接
question: 30.散客旅游
question: 31.广义的导
question: 32.导游讲解
question: 33.导游服务
question: 34.因为游客
question: 35.导游在带
question: 36.游客若在
没找到.或• 但匹配到题号 37
question: 37.如果旅游
question: 38.台风是世
question: 39.导游带团
question: 40.旅客自

question: 41.2004
question: 42.《宪法》
question: 43.宪法的修
question: 44.根据我国
question: 45.我国宪法
question: 46.《旅游法
question: 47.根据《旅
question: 48.根据《旅
question: 49.根据《旅
question: 50.根据《民
question: 51.《民法典
question: 52.根据《民
question: 53.根据《民
question: 54.